In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from downloader import download_all
from preprocess import preprocess_files
from process_headers import process_headers, value_fixes

import dotenv
dotenv.load_dotenv('.env', override=True)

True

In [3]:
filenames = download_all()

In [4]:
data = process_headers(preprocess_files(filenames)).results()[0][0]
# data[:200]

1999 downloads/lamas-muni-1999.xls
2000 downloads/lamas-muni-2000.xls
2001 downloads/lamas-muni-2001.xls
2002 downloads/lamas-muni-2002.xls
2003 downloads/lamas-muni-2003.xls
2004 downloads/lamas-muni-2004.xls
2005 downloads/lamas-muni-2005.xls
2006 downloads/lamas-muni-2006.xls
2007 downloads/lamas-muni-2007.xls
BAD FLOATS {'x'}
BAD FLOATS {'x'}
2008 downloads/lamas-muni-2008.xls
BAD FLOATS {'x'}
BAD FLOATS {'x'}
2009 downloads/lamas-muni-2009.xls
BAD FLOATS {'x'}
BAD FLOATS {'x'}
2010 downloads/lamas-muni-2010.xls
BAD FLOATS {'x'}
BAD FLOATS {'x'}
2011 downloads/lamas-muni-2011.xls
BAD FLOATS {'x'}
BAD FLOATS {'x'}
2012 downloads/lamas-muni-2012.xls
2013 downloads/lamas-muni-2013.xls
2014 downloads/lamas-muni-2014.xls
BAD FLOATS {'10.000.0', '144.000.0', '56.862.0', '11.000.0', '1777.000.0', '447.000.0', '569.453.0', '2684.000.0', '21.000.0', '3.000.0', '232.000.0', '54.000.0'}
2015 downloads/lamas-muni-2015.xls
BAD FLOATS {'2.930.0', '573.865.0', '2165.000.0', '2782.000.0', '11.000.

/Users/adam/.pyenv/versions/3.9.4/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


BAD FLOATS {'=Q94/R94100'}
2018 downloads/lamas-muni-2018.xlsx
2019 downloads/lamas-muni-2019.xlsx
2020 downloads/lamas-muni-2020.xlsx
ALL HEB YEARS {'תשע"א', 'תשע"ח', 'תשס"ז', 'תשס"ח', 'תשס"ה', 'תש"ף', 'תשע"ו', 'תש"ע', 'תשפ"א', 'תשע"ז', 'תשס"ט', 'תשס"ג', 'תשע"ה', 'תשס"ו', 'תשס"ב', 'תשס"ד', 'תשע"ד', 'תש"ס', 'תשע"ג', 'תשע"ב', 'תשע"ט', 'תשס"א'}
GOT 2821 translations
# HEADERS 1709
0 headers
100 headers
200 headers
300 headers
400 headers
500 headers
600 headers
700 headers
800 headers
900 headers
1000 headers
1100 headers
1200 headers
1300 headers
1400 headers
1500 headers
1600 headers
1700 headers
HEADERS MAP 309


In [5]:
if False:
    import dataflows as DF
    from collections import Counter
    from dataflows_airtable import load_from_airtable, dump_to_airtable, AIRTABLE_ID_FIELD

    print('GETTING MAPPING', len(data))
    mapping = DF.Flow(
        data,
        DF.select_fields(['header', 'orig_header']),
        DF.update_resource(-1, name='headers'),
        DF.join_with_self('headers', ['orig_header', 'header'], dict(header=None, orig_header=None)),
    ).results()[0][0]
    print('GOT MAPPING', len(mapping))

    DF.Flow(
        load_from_airtable('apptGe94qTaLjk5Fr', 'Header Mapping', 'Grid view', 'env://AIRTABLE_API_KEY'),
        DF.select_fields([AIRTABLE_ID_FIELD, 'header', 'orig_header']),
        DF.update_resource(-1, name='current'),
        mapping,
        DF.update_resource(-1, name='headers', path='data/headers.csv'),
        DF.add_field('count', 'integer', 1),
        DF.join('current', ['orig_header'], 'headers', ['orig_header'], {AIRTABLE_ID_FIELD:None}, mode='full-outer'),
        DF.set_type('count', transform=lambda v: 1 if v else 0),
        dump_to_airtable({
            ('apptGe94qTaLjk5Fr', 'Header Mapping'): {
                'resource-name': 'headers',
            }
        }, 'env://AIRTABLE_API_KEY'),
    ).process()


In [6]:
import dataflows as DF
from collections import Counter
from dataflows_airtable import load_from_airtable, dump_to_airtable, AIRTABLE_ID_FIELD

mapping = DF.Flow(
    data,
    DF.select_fields(['header', 'year']),
    DF.update_resource(-1, name='headers'),
    DF.join_with_self('headers', ['header', 'year'], dict(
        header=None,
        year=None,
    )),
    DF.sort_rows('{year}'),
    DF.join_with_self('headers', ['header'], dict(
        header=None,
        years=dict(name='year', aggregate='array'),
        count=dict(aggregate='count'),
        max_year=dict(name='year', aggregate='max'),
        min_year=dict(name='year', aggregate='min')
    )),
).results()[0][0]
print(mapping[:10])

DF.Flow(
    load_from_airtable('apptGe94qTaLjk5Fr', 'Stats', 'Grid view', 'env://AIRTABLE_API_KEY'),
    DF.select_fields([AIRTABLE_ID_FIELD, 'header']),
    DF.update_resource(-1, name='current'),
    mapping,
    DF.update_resource(-1, name='headers', path='data/headers.csv'),
    DF.join('current', ['header'], 'headers', ['header'], {AIRTABLE_ID_FIELD:None}, mode='full-outer'),
    DF.set_type('count', transform=lambda v: v or 0),
    DF.set_type('years', type='string', transform=lambda v: ','.join(sorted(map(str,v))) if v else ''),
    dump_to_airtable({
        ('apptGe94qTaLjk5Fr', 'Stats'): {
            'resource-name': 'headers',
        }
    }, 'env://AIRTABLE_API_KEY'),
).process()


/Users/adam/.pyenv/versions/3.9.4/lib/python3.9/site-packages/dataflows/processors/join.py:189: UserWarning: For the `join` processor the `full=True` flag is deprecated. Please use the "mode" parameter instead.
  warnings.warn(


[{'header': 'אורך חוף מוכרז לרחצה במטרים', 'count': 1, 'max_year': 2004, 'min_year': 2004, 'years': [2004]}, {'header': 'אזור', 'count': 6, 'max_year': 2004, 'min_year': 1999, 'years': [1999, 2000, 2001, 2002, 2003, 2004]}, {'header': 'אחוז בני 0-17 באוכלוסייה', 'count': 22, 'max_year': 2020, 'min_year': 1999, 'years': [1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]}, {'header': 'אחוז בני 0-4 באוכלוסייה', 'count': 22, 'max_year': 2020, 'min_year': 1999, 'years': [1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]}, {'header': 'אחוז בני 10-14 באוכלוסייה', 'count': 22, 'max_year': 2020, 'min_year': 1999, 'years': [1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]}, {'header': 'אחוז בני 15-19 באוכלוסייה', 'count': 22, 'max_year': 2020, 'min_year

(<datapackage.package.Package at 0x12cde7f40>, {})

In [7]:
### WRITE TO DB

import dataflows as DF

DF.Flow(
    data,
    value_fixes(),
    # DF.set_type('orig_header', type='string', transform=lambda v: '/'.join(v)),
    DF.update_resource(-1, name='lamas'),
    DF.dump_to_sql(dict(
        lamas_muni={
            'resource-name': 'lamas',
            'mode': 'rewrite',
            'indexes_fields': [
                ['header'], ['year'], ['name'], ['name', 'header'], ['name', 'year'], ['year', 'header'], ['name', 'year', 'header'],
            ]
        }
    ))
).process()


(<datapackage.package.Package at 0x13d6f9280>,
 {'count_of_rows': 1691228,
  'bytes': None,
  'hash': '26e1f7cbb6325aa6c94fc0a1ac4cc9de',
  'dataset_name': None})